### Model
| model |
| --- |
| gpt-3.5-turbo-0125 |
| gpt-4-0125-preview |

In [ ]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0.1
)

In [ ]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You vre a geography expertc. And you only reply in {language}"),
    AIMessage(content="Hi my name is {name}!"),
    HumanMessage(content="서울과 도쿄의 거리는 얼마인가요. 너의 이름은 뭐니?"),
]

chat.predict_messages(messages)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0.1
)

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "Hi my name is {name}!"),
        ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?"),
    ]
)

prompt = template.format_messages(
    language = "Korean",
    name = "AiGENDA",
    country_a = "Korea",
    country_b = "USA"
)

chat.predict_messages(prompt).content

### 3.3 OutputParser and LCEL 

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        
        return list(map(str.strip, items))

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0
)

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else. And you only reply in {language}."),
        ("ai", "Hi my name is {name}!"),
        ("human", "{question}"),
    ]
)

prompt = template.format_messages(
    name = "aigenda",
    language = "Korean",
    max_items = 10,
    question = "2024 한국 축구 국가대표 맴버",
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else. And you only reply in {language}."),
        ("ai", "Hi my name is {name}!"),
        ("human", "{question}"),
    ]
)

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0
)

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        
        return list(map(str.strip, items))

chain = template | chat | CommaOutputParser()    

chain.invoke({
    "name":"aigenda",
    "language":"Korean",
    "max_items":5,
    "question":"2024 한국 축구 국가대표 맴버",
})

### 3.4 Chaining Chains

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0.1,
    streaming=True,
    callbacks = [StreamingStdOutCallbackHandler()]
)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. \
         Everything you are asked will be answered with a comma separated list of max 5 in lowercase. \
         Do NOT reply with anything else."),
        # ("human", "{question}"),
        ("human", "I want to cook {cuisine} food."),
    ]
)

veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. \
         You find alternative ingredients and explain their preparation. \
         You don't radically modify the recipe. \
         If there is no alternative for a food just say you don't know how to replace it. \
         And you only reply in Korean"),
        ("human", "{recipe}")
    ]
)

chef_chain = chef_prompt | chat

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine":"korea"})



## Model IO

https://python.langchain.com/docs/modules/model_io/

https://python.langchain.com/docs/modules/data_connection/

### 4.1 FewShotPromptTemplate

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0.1,
    streaming=True,
    callbacks = [StreamingStdOutCallbackHandler()]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: 파리
        Language: 불어
        Food: 와인, 치즈
        Currency: 유로화
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: 로마
        Language: 이태리어
        Food: 피자, 파스타
        Currency: 유로화
        """
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: 아테네
        Language: 그리스어
        Food: 수블라키 and 페타 치즈
        Currency: 유로화
        """
    },
]

example_prompt = PromptTemplate.from_template("Human : {question}\nAI : {answer}")
    
prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
    suffix = "Human : What do you know about {country}?",
    input_variables=["country"],
)

chain = prompt | chat

chain.invoke({
    "country" : "아이슬란드"
})

### 4.2 FewShotChatMessagePromptTemplate 

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0.1,
    streaming=True,
    callbacks = [StreamingStdOutCallbackHandler()]
)

examples = [
    {
        "country": "France?",
        "answer": """
        Here is what I know:
        Capital: 파리
        Language: 불어
        Food: 와인, 치즈
        Currency: 유로화
        """,
    },
    {
        "country": "Italy?",
        "answer": """
        I know this:
        Capital: 로마
        Language: 이태리어
        Food: 피자, 파스타
        Currency: 유로화
        """
    },
    {
        "country": "Greece?",
        "answer": """
        I know this:
        Capital: 아테네
        Language: 그리스어
        Food: 수블라키 and 페타 치즈
        Currency: 유로화
        """
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

# 예시 문장과 실제 예시를 보여줌   
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# system : 전달 인자, example_prompt : 예시 문장과 실제 예시 프롬프트, human : 물어볼 메시지
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

print(f"3 : {final_prompt}")

chain = final_prompt | chat

chain.invoke({"country": "Thailand"})

### 4.3 LengthBasedExampleSelector

example의 수를 제한해서 비용을 줄일 수 있는 방법

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0.1,
    streaming=True,
    callbacks = [StreamingStdOutCallbackHandler()],
)

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: 파리
        Language: 불어
        Food: 와인, 치즈
        Currency: 유로화
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: 로마
        Language: 이태리어
        Food: 피자, 파스타
        Currency: 유로화
        """
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: 아테네
        Language: 그리스어
        Food: 수블라키 and 페타 치즈
        Currency: 유로화
        """
    },
]

example_prompt = PromptTemplate.from_template("Human : {question}\nAI : {answer}")

example_selector = RandomExampleSelector(
    examples = examples,
)
    
prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector = example_selector,
    suffix = "Human : What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="브라질")

### 4.4 Serialization and Composition

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.json")

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0.1,
    streaming=True,
    callbacks = [StreamingStdOutCallbackHandler()],
)

prompt.format(country="xxx")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
    """
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
    """
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

### 4.5 Caching

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)


chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("How do you make italian pizza")

### 4.6 Serialization

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

# from langchain.globals import set_llm_cache, set_debug
# from langchain.cache import SQLiteCache

# set_llm_cache(SQLiteCache("cache.db"))
# set_debug(True)

chat = ChatOpenAI(
    # model_name = "gpt-4-0125-preview",
    model_name="gpt-3.5-turbo-0125",
    temperature=0.1,
    max_tokens= 1000,
)


with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for toast")
    b = chat.predict("What is the recipe for egg sandwich")
    print(a, "\n")
    print(b, "\n")
    print(usage)

## 5 MEMORY
### 5.0 ConversationBufferMemory

전체 메모리를 저장하는 방법

단점 : 비용이 많이 듬

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input":"Hi!"},{"output":"How are you?"})

memory.load_memory_variables({})

### ConversationBufferWindowMemory

최근 메모리만을 저장하는 방법

단점 : 최근 대화에만 집중함

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4,
)

def add_message(input, output):
    memory.save_context({"input":input},{"output":output})

add_message(1, 1)

In [ ]:
add_message(2,2)
add_message(3,3)
add_message(4,4)
add_message(5,5)

In [ ]:
add_message(5,5)

In [ ]:
memory.load_memory_variables({})

## 5.2 ConversationSummaryMemory 

llm을 사용하여 요약하여 메모리에 저장

장점 : 매우 긴 대화에서 간편하게 메모리를 저장할 수 있음

단점 : 메모리를 실행하기 위해 llm을 동작해야함


In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model_name = "gpt-3.5-turbo-0125",
    temperature=0.1,
)

memory = ConversationSummaryMemory(llm=llm)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

In [ ]:
add_message("South Kddorea is so pretty", "I wish I could go!!!")

In [ ]:
get_history()

### 5.3 ConversationSummaryBufferMemory

Conversation Summary Memory + Conversation Buffer Memory

메모리에 보내온 메시지의 수를 저장

최근 메시지는 그대로 저장

리미트에 다달한 메시지는 요약하여 저장함


In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    model_name = "gpt-3.5-turbo-0125",
    llm=llm,
    max_token_limit=150,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

In [ ]:
get_history()

In [ ]:
add_message("South Korea is so pretty", "I wish I could go!!!")

In [ ]:
get_history()

In [ ]:
add_message("How far is Korea from Argentina?", "I don't know! Super far!")
add_message("How far is Brazil from Argentina?", "I don't know! Super far!")

In [ ]:
get_history()

### 5.4 ConversationKGMemory 

Conversation Knowledge Graph Memory

Knowledge Graph를 뽑아내어 요약

In [ ]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")
add_message("He is Hyoin, he from Korean", "Wow that is so cool!")

In [ ]:
memory.load_memory_variables({"input": "who is Hyoin"})

In [ ]:
add_message("He likes hamberger.", "Wow that is so cool!")

In [ ]:
memory.load_memory_variables({"input": "What does Hyoin like?"})

### 5.5 Memory on LLMChain 

메모리와 LLM을 연결하여 작동

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",   
    temperature=0.1
)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=80,
    memory_key="chat_history",
)


template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

chain.predict(question="My name is Nico")

In [ ]:
chain.predict(question="I live in Seoul")

In [ ]:
chain.predict(question="where am I")

In [ ]:
chain.predict(question="I'm 27")

In [ ]:
chain.predict(question = "How old am I?")

### 5.6 Chat Based Memory

실제 채팅이 가능한 메모리로 바꿈

기존에 코드는 메모리를 string으로 저장함

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",   
    temperature=0.1
)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=80,
    memory_key="chat_history",
    return_messages = True,
)

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful AI talking to a human."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human","{question}")
])

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

chain.predict(question="My name is Nico")

In [ ]:
chain.predict(question="I live in Seoul")
chain.predict(question="I'm 27")

In [ ]:
chain.predict(question="Qize Time! What is my name? And How old am I?")

### 5.7 LCEL Based Memory

LangChain Expression Language

메모리에 쉽게 추가하는 방법

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",   
    temperature=0.1
)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=80,
    return_messages = True,
)

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful AI talking to a human."),
    MessagesPlaceholder(variable_name="history"),
    ("human","{question}")
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

# invoke_chain("My name is eggsy")

In [ ]:
invoke_chain("My name is eggsy")

## 6 RAG

Retrieval Augumented Generation(검색 증강 생성)

질문을 개인의 문서와 함께 prompt에서 합치는 방법

<img src="https://python.langchain.com/assets/images/data_connection-95ff2033a8faa5f3ba41376c0f6dd32a.jpg">

Load : 어떤 소스에서 데이터를 받아옴

Transform : 받아온 데이터를 분할

Embed : 데이터를 컴퓨터가 알아듣기 쉽게 벡터화

Store : 벡터 데이터를 저장 (Vector Store)

### 6.1 Data Loaders and Splitters

RAG의 첫번째 단계 : 데이터 검색

<https://python.langchain.com/docs/modules/data_connection/document_loaders/>

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

# splitter = RecursiveCharacterTextSplitter(
#     chunk_size=200,
#     chunk_overlap=50
# )

splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
    length_function=len,
)

loader = UnstructuredFileLoader("./files/chapter_one.pdf")

loader.load_and_split(text_splitter=splitter)

### 6.2 Tiktoken
<https://platform.openai.com/tokenizer>

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)


loader = UnstructuredFileLoader("./files/chapter_one.docx")


### 6.3 Vectors

Embedding : 사람이 읽는 텍스트를 컴퓨터가 쉽게 이해할 수 있도록 변환하는 과정

각각의 문서를 Vectorization화함

연습에서는 3차원 백터로 단어를 표현

openAI에서는 1536차원의 벡터로 단어를 

<https://turbomaze.github.io/word2vecjson/>

<https://www.youtube.com/watch?v=2eWuYf-aZE4>

Masculinity | Femininity | Royalty

king    | 0.9   | 0.1   | 1.0
queen   | 0.1   | 0.9   | 1.0
man     | 0.9   | 0.1   | 0.0
woman   | 0.1   | 0.9   | 0.0

royal = king - man | 0.0    | 0.0   | 1.0
queen = royal + woman   | 0.1   | 0.9   | 1.0

### 6.4 Vector Store

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

embedder = OpenAIEmbeddings()

vector = embedder.embed_documents([
    "hi",
    "how",
    "are",
    "you",
    "longer sentences because"
])

len(vector[4])

In [ ]:
# Chroma를 이용하여 임배딩
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")
# loader = UnstructuredFileLoader("./files/lucky_day.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embedding = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir,
)

vectorstore = Chroma.from_documents(docs, cached_embedding)

In [ ]:
results = vectorstore.similarity_search("주인공은 왜 기분이 좋은가?")

results
# len(results)

### 6.5 Langsmith

<https://www.langchain.com/langsmith>

### 6.6 RetrievalQA

생성한 문서를 이용하여 LLM에게 질문함

off-the-shelf chain : 미리 만들어 놓은 chain을 사용함

#### Stuff document chain Method
<https://js.langchain.com/docs/modules/chains/document/stuff>

<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*GykpE5AdrPkT2jPM3XH-QQ.png">

docs와 question을 prompt에 묶어서 한번에 질문

---

#### Refine document chain
<https://js.langchain.com/docs/modules/chains/document/refine>

<img src="https://js.langchain.com/assets/images/refine-a70f30dd7ada6fe5e3fcc40dd70de037.jpg">

여러개의 document를 순차적으로 질문하고 이를 반복하면서 정제된 답변을 도출

Stuff document chain 보다 더 비쌈

---

#### Map reduce document chain
<https://js.langchain.com/docs/modules/chains/document/map_reduce>

<img src="https://js.langchain.com/assets/images/map_reduce-c65525a871b62f5cacef431625c4d133.jpg">

document를 입력받아서, 개별적으로 요약작업 수행

각각의 요약본을 LLM에 전달

매우 많은 연산작업이 필요

---

### Map re-rank document chain (현재 안됨)
<https://python.langchain.com.cn/docs/modules/chains/document/map_rerank>

<img src="https://python.langchain.com.cn/assets/images/map_rerank-0302b59b690c680ad6099b7bfe6d9fe5.jpg">

각 document를 입력받아서, 개별적으로 요약작업을 수행하고 점수(score)를 매김

가장 높은 점수인 답변을 점수와 함께 도출

In [ ]:
# vectorstore : Chroma 방식
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")
# loader = UnstructuredFileLoader("./files/lucky_day.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embedding = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir,
)

vectorstore = Chroma.from_documents(docs, cached_embedding)

chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)

In [ ]:
chain.run("Who is a main character. Tell me by Korean")

In [ ]:
# vectorstore : FAISS
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

# loader = UnstructuredFileLoader("./files/chapter_one.docx")
loader = UnstructuredFileLoader("./files/lucky_day.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embedding = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir,
)

vectorstore = FAISS.from_documents(docs, cached_embedding)

chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type="refine",
    retriever=vectorstore.as_retriever(),
)

In [ ]:
chain.run("Who is a main character. Tell me by Korean")

In [ ]:
chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type="map_reduce",
    retriever=vectorstore.as_retriever(),
)

chain.run("여기에 등장하는 인물 모두 말해줘.")

In [ ]:
chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type="map_reduce",
    retriever=vectorstore.as_retriever(),
)

chain.run("Describe Victory Mansions")

### 6.8 Stuff LCEL Chain

LangChain Expression Language를 사용해서 stuff chain을 구현

In [7]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")
# loader = UnstructuredFileLoader("./files/lucky_day.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embedding = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir,)

vectorstore = FAISS.from_documents(docs, cached_embedding)

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature=0.1
)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a helpful assistant. \
        Answer questions using only the following context. \
        If you don't know the answer just say you don't know, \
        don't make it up:\n\n{context}",
    ),
    ("human", "{question}",),
])

chain = {"context":retriver, "question":RunnablePassthrough()} | prompt | llm

chain.invoke("Describe Victory Mansions.")

AIMessage(content="Victory Mansions is a building with glass doors that Winston Smith enters. The hallway smells of boiled cabbage and old rag mats. It has seven flights of stairs, and the lift is seldom working. There is a large colored poster of a man's face on the wall, and the building is not well maintained, with the electric current cut off during daylight hours as part of an economy drive.")

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature=0.1
)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a helpful assistant. \
        Answer questions using only the following context. \
        If you don't know the answer just say you don't know, \
        don't make it up:\n\n{context}",
    ),
    ("human", "{question}",),
])

chain = (
    {
        "context":retriver,
        "question":RunnablePassthrough(),
    }
    | prompt
    | llm
)

chain.invoke("who is main character")

AIMessage(content='The main character is Winston Smith.')

### 6.9 Map Reduce LCEL Chain

LangChain Expression Language를 사용해서 Map Reduce chain을 구현

In [1]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")
# loader = UnstructuredFileLoader("./files/lucky_day.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embedding = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir,)

vectorstore = FAISS.from_documents(docs, cached_embedding)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature=0.1
)

retriver = vectorstore.as_retriever()

# 1 : list of docs
# 2 : for doc in list of docs | prompt | llm
# 3 : for respone in list of llms respons | put them all together
# 4 : final doc | prompt | llm 

map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question.
            Return any relevant text verbatim.
            If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

# 2. for doc in list of docs | prompt | llm
map_doc_chain = map_doc_prompt | llm

# 3. for respone in list of llms respons | put them all together
def map_docs(inputs):
    documents = inputs['documents']
    question = inputs['question']
    # results = []
    # for document in documents:
    #     result = map_doc_chain.invoke({
    #         "context": document.page_content,
    #         "question": question
    #     }).content
    #     results.append(result)
    # return "\n\n".join(results)
    return "\n\n".join(
        map_doc_chain.invoke(
            {"context": doc.page_content, "question": question}
        ).content
        for doc in documents
    )

# 1. list of docs
map_chain = {"documents":retriver, "question": RunnablePassthrough()} | RunnableLambda(map_docs)
    
# 4. final doc | prompt | llm 
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}")
    ]
)

chain = (
    {
        "context": map_chain,
        "question": RunnablePassthrough()
    }
    | final_prompt
    | llm
)

chain.invoke("Describe Vitory Mansions")
# chain.invoke("Where does Winston go to work?")

AIMessage(content='Winston goes to work at the Ministry of Truth, which is described as an enormous pyramidal structure of glittering white concrete, soaring up 300 meters into the air.')